In [1]:
from cosecha_colectiva import miscelaneous as m
from cosecha_colectiva import config as config
m.try_find_conf_file()


from cosecha_colectiva import miscelaneous as m

import logging
import pathlib
from datetime import datetime
import datajoint as dj
import time
import numpy as np

import pandas as pd


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
def number_to_excel_column(n):
    result = ''
    while n > 0:
        remainder = (n - 1) % 26
        result = chr(ord('A') + remainder) + result
        n = (n - 1) // 26
    return result

In [3]:
cosecha_db = dj.create_virtual_module(config.db_name, config.db_name)

[2025-05-07 21:14:06,579][INFO]: Connecting root@junction.proxy.rlwy.net:19994
[2025-05-07 21:14:08,116][INFO]: Connected root@junction.proxy.rlwy.net:19994


## Ingresa datos de Grupos

In [4]:
ogsd = cosecha_db.OldGruposStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ogsd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

'A:BN'

In [5]:
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStats',
                          skiprows=30, nrows=1, names=this_cols, header=None, dtype=this_dtypes
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ogsd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)


0
1


## Lee datos para las otras tablas (num_socios, num_sesiones, nombre_grupo)

In [6]:
num_socios_grupo = [] 
Grupos_Names = []
num_sesiones_grupo = []
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    data_num_socios = pd.read_excel(url,
                          sheet_name='DatosParaStats',
                        nrows=3, usecols='A:B', header=None
                  )
    num_socios_grupo.append(int(data_num_socios.loc[0,1]))
    Grupos_Names.append(data_num_socios.loc[1,1])
    data_num_sesiones = pd.read_excel(url,
                          sheet_name='DatosParaStats',
                          skiprows=59, nrows=1, usecols='A:C', header=None
                  )
    num_sesiones_grupo.append(int(data_num_sesiones.loc[0,2]))


0
1


## Ingresa datos Socios

In [7]:
ossd = cosecha_db.OldSociosStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ossd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

'A:BI'

In [8]:
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[i])
    print('num_socios_grupo[i]', num_socios_grupo[i])
    print('num_sesiones_grupo[i]', num_sesiones_grupo[i])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStats',
                          skiprows=1, nrows=num_socios_grupo[i], names=this_cols, usecols=columns_read, dtype=this_dtypes, header=None
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ossd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)


0
La Luz
num_socios_grupo[i] 12
num_sesiones_grupo[i] 93
1
ABEJAS
num_socios_grupo[i] 11
num_sesiones_grupo[i] 78


## Ingresa datos Sesiones

In [9]:
oSeSd = cosecha_db.OldSesionesStatsDashboard()
table_attributes = pd.DataFrame.from_dict(oSeSd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'D:'+number_to_excel_column(len(this_cols)+3)
columns_read

'D:BM'

In [12]:
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[i])
    print('num_socios_grupo[i]', num_socios_grupo[i])
    print('num_sesiones_grupo[i]', num_sesiones_grupo[i])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStatsSesiones',
                          skiprows=200, nrows=num_sesiones_grupo[i], names=this_cols, usecols=columns_read, header=None, dtype=this_dtypes
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    oSeSd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)


0
La Luz
num_socios_grupo[i] 12
num_sesiones_grupo[i] 93
1
ABEJAS
num_socios_grupo[i] 11
num_sesiones_grupo[i] 78


In [13]:
df_data

,nombre_grupo,num_sesion,Sesion_id,Fecha,Activa,Caja,Acciones,Ganancias,Fecha_prox_reunion,Lugar_prox_reunion,...,sum_ganancias,sum_ganancias_entregadas,num_ganancias,sesion_id_interes_prestamo,sum_interes,num_prestamos_activos,num_prestamos_vigentes,num_prestamos_morosos,num_prestamos_ampliados,num_prestamos_ampliados_morosos
0,ABEJAS,1,200,2018-01-01,0,None,950,10.0,2018-02-01,None,...,10.0,10.0,9,200,2.0,2,2,0,0,0
1,ABEJAS,2,201,2018-02-01,0,None,910,36.0,2018-03-01,None,...,36.0,36.0,9,201,5.0,5,5,0,0,0
2,ABEJAS,3,202,2018-03-01,0,None,1150,46.0,2018-04-01,None,...,46.0,46.0,9,202,5.0,5,5,0,0,0
3,ABEJAS,4,203,2018-04-01,0,None,1150,69.5,2018-05-01,None,...,69.5,69.5,9,203,6.0,6,6,0,0,0
4,ABEJAS,5,204,2018-05-01,0,None,610,60.0,2018-06-01,None,...,60.0,60.0,9,204,5.0,5,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,ABEJAS,74,273,2024-08-01,0,None,200,595.0,2024-09-01,None,...,595.0,595.0,11,273,6.0,6,6,0,0,0
74,ABEJAS,75,274,2024-09-01,0,None,800,1279.0,2024-10-01,None,...,1279.0,1279.0,11,274,9.0,9,9,0,0,0
75,ABEJAS,76,275,2024-10-01,0,None,1100,807.5,2024-11-01,None,...,807.5,807.5,11,275,8.0,8,8,0,0,0
76,ABEJAS,77,276,2024-11-01,0,None,900,829.5,2024-12-01,None,...,829.5,829.5,11,276,8.0,8,8,0,0,0
